In [1]:
from confluent_kafka import Consumer, KafkaException

In [2]:
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'CountryCounter',
    'auto.offset.reset': 'earliest'
}

In [3]:
consumer = Consumer(conf)

In [7]:
def on_assign(consumer, partitions):
    print("Partitions assigned:", partitions)
    # Optionally, you can seek to a specific offset
    for partition in partitions:
        partition.offset = 0
    consumer.assign(partitions)

consumer.subscribe(['my_topic'], on_assign=on_assign)

In [8]:
try:
    while True:
        msg = consumer.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition event
                print(f"Reached end of partition: {msg.topic()} [{msg.partition()}] at offset {msg.offset()}")
            elif msg.error():
                raise KafkaException(msg.error())
        else:
            # Proper message
            print(f"Received message: {msg.value().decode('utf-8')}")

except KeyboardInterrupt:
    pass
finally:
    # Close the consumer
    consumer.close()


Partitions assigned: [TopicPartition{topic=my_topic,partition=%I32d,offset=%s,leader_epoch=%s,error=%s}, TopicPartition{topic=my_topic,partition=%I32d,offset=%s,leader_epoch=%s,error=%s}, TopicPartition{topic=my_topic,partition=%I32d,offset=%s,leader_epoch=%s,error=%s}]
Received message: Apple message
Received message: Banana message


: 